ANN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Read the data
train_data = pd.read_excel('Train Set.xlsx', sheet_name='Interiors')
test_data = pd.read_excel('Validation Set.xlsx', sheet_name='Interiors')
val_data = pd.read_excel('Test Set.xlsx',sheet_name='Interiors')

X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
X_val = val_data.iloc[:, :-1]
y_val = val_data.iloc[:, -1]

# Create a LabelEncoder instance
encoder = LabelEncoder()

# Fit the encoder to the training data and then transform the training, validation, and test data
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
y_val = encoder.transform(y_val)

from sklearn.preprocessing import StandardScaler
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data and then transform the training, validation, and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Convert DataFrame to Tensor
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float)
y_val = torch.tensor(y_val, dtype=torch.long)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(9, 11),  # First layer
            nn.Tanh(),
            nn.Linear(11, 22),
            nn.Sigmoid(),
            nn.Linear(22, 6),  # Third layer
        )

    def forward(self, x):
        x = self.layers(x)
        return x

net = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

best_acc = 0.0
best_model_state = None

# Use the validation set to tune the model hyperparameters
net = Net()  
optimizer = optim.AdamW(net.parameters(), lr=1e-2, weight_decay=0.1)  # Optimizer needs to be reinitialized as well

for epoch in range(1000):  # loop over the dataset multiple times
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # print statistics
    if epoch % 10 == 0:
        val_outputs = net(X_val)
        _, predicted = torch.max(val_outputs.data, 1)
        correct = (predicted == y_val).type(torch.float).sum().item()
        accuracy = correct / len(y_val)

        if accuracy > best_acc:
            best_acc = accuracy
            best_model_state = net.state_dict()

        if accuracy > 0.7:
            print('Early stopping at epoch: ', epoch)
            break

# Load the model with the highest accuracy
net.load_state_dict(best_model_state)

# Compute the accuracy
train_outputs = net(X_train)
_, predicted = torch.max(train_outputs.data, 1)
train_acc = accuracy_score(y_train, predicted)

test_outputs = net(X_test)                        
_, predicted = torch.max(test_outputs.data, 1)
test_acc = accuracy_score(y_test, predicted)

val_outputs = net(X_val)
_, predicted = torch.max(val_outputs.data, 1)
val_acc = accuracy_score(y_val, predicted)

print('Train Accuracy: ', train_acc)
print('Validation Accuracy: ', val_acc)
print('Test Accuracy: ', test_acc)


# Print true and predicted labels for the validation set
print('Validation True: ', encoder.inverse_transform(y_val))
print('Validation Predicted: ', encoder.inverse_transform(predicted.numpy()))

# Print true and predicted labels for the test set
print('Test True: ', encoder.inverse_transform(y_test))
print('Test Predicted: ', encoder.inverse_transform(predicted.numpy()))


SVM

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the data
train_data = pd.read_excel('Train Set.xlsx', sheet_name='Interiors')
test_data = pd.read_excel('Validation Set.xlsx', sheet_name='Interiors')
val_data = pd.read_excel('Test Set.xlsx',sheet_name='Interiors')

X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_val = val_data.iloc[:, :-1]
y_val = val_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

# Create a LabelEncoder instance
encoder = LabelEncoder()

# Fit the encoder to the training data and then transform the training, validation, and test data
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data and then transform the training, validation, and test data
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

from sklearn.decomposition import PCA

# Create a PCA instance
pca = PCA(n_components=3)  # Here we are assuming we are reducing the data to 2 dimensions

# Fit PCA to the training data and then transform the training, validation, and test data
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'] # Here we are specifying four kernel functions
}

# Create an SVM model instance
svm_model = svm.SVC()

# Create a GridSearchCV instance
grid_search = GridSearchCV(svm_model, param_grid, cv=5)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print('Best parameters: ', best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Predict the training set
y_train_pred = best_model.predict(X_train)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print('SVM Model Train Accuracy: {:.2f}%'.format(train_accuracy * 100))

# Predict the validation set
y_val_pred = best_model.predict(X_val)

# Calculate the validation accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print('SVM Model Validation Accuracy: {:.2f}%'.format(val_accuracy * 100))

# Predict the test set
y_test_pred = best_model.predict(X_test)

# Calculate the test accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print('SVM Model Test Accuracy: {:.2f}%'.format(test_accuracy * 100))


XG-Boost

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the data
train_data = pd.read_excel('Train Set.xlsx', sheet_name='Interiors')
test_data = pd.read_excel('Validation Set.xlsx', sheet_name='Interiors')
val_data = pd.read_excel('Test Set.xlsx',sheet_name='Interiors')


X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_val = val_data.iloc[:, :-1]
y_val = val_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

# Create a LabelEncoder instance
encoder = LabelEncoder()

# Fit the encoder to the training data and then transform the training, validation, and test data
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data and then transform the training, validation, and test data
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_depth': [None, 3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2, 0.5]
}

# Create an XGBoost model instance
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Create a GridSearchCV instance
grid_search = GridSearchCV(xgb_model, param_grid, cv=5)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print('Best parameters: ', best_params)

# Get the best model
best_model = grid_search.best_estimator_

# Predict the training set
y_train_pred = best_model.predict(X_train)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print('XGBoost Model Train Accuracy: {:.2f}%'.format(train_accuracy * 100))

# Predict the validation set
y_val_pred = best_model.predict(X_val)

# Calculate the validation accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print('XGBoost Model Validation Accuracy: {:.2f}%'.format(val_accuracy * 100))

# Predict the test set
y_test_pred = best_model.predict(X_test)

# Calculate the test accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print('XGBoost Model Test Accuracy: {:.2f}%'.format(test_accuracy * 100))



CNN-LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import os


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training.")

# Load the data
train_data = pd.read_excel('Train Set.xlsx', sheet_name='Interiors')
test_data = pd.read_excel('Validation Set.xlsx', sheet_name='Interiors')
val_data = pd.read_excel('Test Set.xlsx',sheet_name='Interiors')

X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_val = val_data.iloc[:, :-1]
y_val = val_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

# Label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

# Check for NaN or Inf values
assert not np.any(np.isnan(X_train))
assert not np.any(np.isnan(X_test))
assert not np.any(np.isinf(X_train))
assert not np.any(np.isinf(X_test))

# Convert to tensors
X_train = torch.tensor(X_train.values, dtype=torch.float).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_val = torch.tensor(X_val.values, dtype=torch.float).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

class CNNLSTM(nn.Module):
    def __init__(self, num_samples):
        super(CNNLSTM, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )
        self.fc = nn.Linear(64, 64)
        self.dropout = nn.Dropout(0.2)  # Add Dropout layer
        self.lstm = nn.LSTM(input_size=64, hidden_size=num_samples, num_layers=1, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(9 * 2 * num_samples, 150)
        self.fc2 = nn.Linear(150, 6)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.permute(0, 2, 1)
        x = self.fc(x)
        x = self.dropout(x)  # Apply Dropout layer here
        x, _ = self.lstm(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Get the number of samples in the training set
num_samples = X_train.shape[0]

# Instantiate the CNN-LSTM model with the number of samples
net = CNNLSTM(num_samples).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Training loop
best_val_acc = 0.0
best_model_state = None

for epoch in range(500):
    net.train()  # Switch to train mode
    optimizer = optim.AdamW(net.parameters(), lr=1e-3, weight_decay=0.1)
    optimizer.zero_grad()
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Print statistics
    if epoch % 50 == 0:
        net.eval()  # Switch to evaluate mode
        val_outputs = net(X_val)
        _, predicted = torch.max(val_outputs.data, 1)
        correct = (predicted == y_val).type(torch.float).sum().item()
        accuracy = correct / len(y_val)

        print(f"Epoch {epoch}: Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.4f}")

        if accuracy > best_val_acc:
            best_val_acc = accuracy
            # Save the best model state here
            best_model_state = net.state_dict()

        if accuracy > 0.8:
            print('Early stopping at Epoch: ', epoch)
            break

# Load the best model state after all epochs
if best_model_state is not None:
    net.load_state_dict(best_model_state)

# Evaluate the model
test_outputs = net(X_test)
_, predicted = torch.max(test_outputs.data, 1)
test_acc = accuracy_score(y_test.cpu().numpy(), predicted.cpu().numpy())

print('Test Accuracy: ', test_acc)
print('Test Set Predicted Labels: ', encoder.inverse_transform(predicted.cpu().numpy()))
print('Test Set True Labels: ', encoder.inverse_transform(y_test.cpu().numpy()))

# Validation
train_outputs = net(X_train)
_, predicted = torch.max(train_outputs.data, 1)
train_acc = accuracy_score(y_train.cpu().numpy(), predicted.cpu().numpy())

print('Train Accuracy: ', train_acc)


SHAP

In [ ]:
import shap
import torch
import matplotlib.pyplot as plt


# Load the model state, make sure to load it on CPU
model_state = torch.load('best_model_state.pth', map_location='cpu')

# Apply the model state
net.load_state_dict(model_state)
net.eval()
net.to('cpu')

features = train_data.iloc[:, :-1]

# Explicitly specify the feature names used during training
feature_names = features.columns.tolist()

# Create a function to wrap your model so that it can accept numpy arrays as input and return numpy arrays as output
def predict_fn(x):
    # Convert the input data to PyTorch tensor and make sure to compute on CPU
    x = torch.from_numpy(x).type(torch.FloatTensor)
    with torch.no_grad():
        outputs = net(x)
    return outputs.numpy()

# Make sure X_train is already on CPU
X_train_cpu = X_train.cpu().numpy()

# Create a SHAP explainer
explainer = shap.KernelExplainer(predict_fn, X_train_cpu, feature_names=feature_names)

# Select an instance to explain
i = 11
shap_values = explainer.shap_values(X_test.cpu().numpy()[i])

# Print the instance's values and label
print("Instance values:", [round(num, 2) for num in X_test.cpu().numpy()[i]])
print("Instance label:", encoder.inverse_transform([y_test.cpu().numpy()[i]]))

import numpy as np

# Set numpy printing options to keep 2 decimal places
np.set_printoptions(precision=2, suppress=True)

# Print the shap_values
for class_index, shap_array in enumerate(shap_values):
    print(f"SHAP values for class {class_index}:")
    print(shap_array)

# Create a function to wrap your model so that it can accept numpy arrays as input and return numpy arrays as output
def predict(input):
    # Convert the input data to PyTorch tensor and make sure to compute on CPU
    input_tensor = torch.tensor(input, dtype=torch.float)
    with torch.no_grad():
        output = net(input_tensor)
        _, predicted = torch.max(output.data, 1)
        return predicted.numpy()


# Create a SHAP explainer and associate feature names
explainer = shap.Explainer(predict, X_train_cpu, feature_names=feature_names)

import matplotlib.pyplot as plt

# Make sure X_test and X_val are already on CPU
X_test_cpu = X_test.cpu().numpy()
X_val_cpu = X_val.cpu().numpy()

# Concatenate the test and validation set
X_combined = np.concatenate((X_test_cpu, X_val_cpu), axis=0)

# Calculate SHAP values for the concatenated dataset
shap_values_combined = explainer(X_combined)

import matplotlib.pyplot as plt
import shap

# Create a new figure, set DPI to 600
plt.figure(dpi=600)

# Plot SHAP summary plot on the new figure
shap.summary_plot(shap_values_combined, X_combined, feature_names=feature_names)

# Create another new figure, set DPI to 600
plt.figure(dpi=600)

# Plot SHAP heatmap on the new figure
shap.plots.heatmap(shap_values_combined)

# Display all the plots
plt.show()